In [1]:
import pandas as pd

In [6]:
from tqdm.notebook import tqdm
import sqlite3

In [7]:
db = sqlite3.connect('data/Hop_Teaming_2018.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('Hop_Teaming_2018', db, if_exists = 'append', index = False)            # Append the chunk to a calls table

0it [00:00, ?it/s]

In [12]:
query = "SELECT * FROM Hop_Teaming_2018 LIMIT 5"

with sqlite3.connect('data/Hop_Teaming_2018.sqlite') as db: 
    Hop_Teaming_sqlite = pd.read_sql(query, db)

In [13]:
Hop_Teaming_sqlite

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880


The NPPES dataset contains a large number of fields, only a few of which are relevant to this project:

'NPI'

Entity Type, indicated by the 'Entity Type Code' field:
1 = Provider (doctors, nurses, etc.)
2 = Facility (Hospitals, Urgent Care, Doctors Offices)

Entity Name: Either First/Last or Organization or Other Organization Name contained in the following fields:
'Provider Organization Name (Legal Business Name)'
'Provider Last Name (Legal Name)'
'Provider First Name'
'Provider Middle Name'
'Provider Name Prefix Text'
'Provider Name Suffix Text'
'Provider Credential Text'

Address: Business Practice Location (not mailing), contained in the following fields:
'Provider First Line Business Practice Location Address'
'Provider Second Line Business Practice Location Address'
'Provider Business Practice Location Address City Name'
'Provider Business Practice Location Address State Name'
'Provider Business Practice Location Address Postal Code'

The provider's taxonomy code, which is contained in one of the 'Healthcare Provider Taxonomy Code*' columns. A provider can have up to 15 taxonomy codes, but we want the one which has Primary Switch = Y in the associated 'Healthcare Provider Primary Taxonomy Switch*' field. Note that this does not always occur in spot 1.

In [149]:
db = sqlite3.connect('data/Hop_Teaming_2018.sqlite')

for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',  
                              usecols = ['NPI','Entity Type Code',
                                         'Provider Organization Name (Legal Business Name)',
                                         'Provider Last Name (Legal Name)',
                                         'Provider First Name',
                                         'Provider Middle Name',
                                         'Provider Name Prefix Text',
                                         'Provider Name Suffix Text',
                                         'Provider Credential Text',
                                         'Provider First Line Business Practice Location Address',
                                         'Provider Second Line Business Practice Location Address',
                                         'Provider Business Practice Location Address City Name',
                                         'Provider Business Practice Location Address State Name',
                                         'Provider Business Practice Location Address Postal Code',
                                         'Healthcare Provider Taxonomy Code_1',
                                         'Healthcare Provider Primary Taxonomy Switch_1',
                                         'Healthcare Provider Taxonomy Code_2',
                                         'Healthcare Provider Primary Taxonomy Switch_2',
                                         'Healthcare Provider Taxonomy Code_3',
                                         'Healthcare Provider Primary Taxonomy Switch_3',
                                         'Healthcare Provider Taxonomy Code_4',
                                         'Healthcare Provider Primary Taxonomy Switch_4',
                                         'Healthcare Provider Taxonomy Code_5',
                                         'Healthcare Provider Primary Taxonomy Switch_5',
                                         'Healthcare Provider Taxonomy Code_6',
                                         'Healthcare Provider Primary Taxonomy Switch_6',
                                         'Healthcare Provider Taxonomy Code_7',
                                         'Healthcare Provider Primary Taxonomy Switch_7',
                                         'Healthcare Provider Taxonomy Code_8',
                                         'Healthcare Provider Primary Taxonomy Switch_8',
                                         'Healthcare Provider Taxonomy Code_9',
                                         'Healthcare Provider Primary Taxonomy Switch_9',
                                         'Healthcare Provider Taxonomy Code_10',
                                         'Healthcare Provider Primary Taxonomy Switch_10',
                                         'Healthcare Provider Taxonomy Code_11',
                                         'Healthcare Provider Primary Taxonomy Switch_11',
                                         'Healthcare Provider Taxonomy Code_12',
                                         'Healthcare Provider Primary Taxonomy Switch_12',
                                         'Healthcare Provider Taxonomy Code_13',
                                         'Healthcare Provider Primary Taxonomy Switch_13',
                                         'Healthcare Provider Taxonomy Code_14',
                                         'Healthcare Provider Primary Taxonomy Switch_14',
                                         'Healthcare Provider Taxonomy Code_15',
                                         'Healthcare Provider Primary Taxonomy Switch_15',],
                              chunksize = 10000)):
  
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk[ 'primary_taxonomy' ] = ""
    for index, row in chunk.iterrows():         #code below create a list of taxonomy values IF the row has more than one 'Y'
        chunk.loc[index, 'primary_taxonomy'] = ([ row[f'healthcare_provider_taxonomy_code_{n}'] for n in range(1,16) \
            if row[ f'healthcare_provider_primary_taxonomy_switch_{n}' ] == 'Y' ]+["none"])[0] #some rows has no switch = 'Y', 
                                                                            #which results in an empty list
    chunk.drop( columns=[ f'healthcare_provider_taxonomy_code_{n}' for n in range(1,16) ], inplace = True )
    chunk.drop( columns=[ f'healthcare_provider_primary_taxonomy_switch_{n}' for n in range(1,16) ], inplace = True )
    chunk = chunk[ chunk['primary_taxonomy'] != 'none' ]
    chunk.to_sql('nppes', db, if_exists = 'append', index = False) 
    #print(chunk)
    #break
    
 

0it [00:00, ?it/s]

/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (71,74,75,78,79,82,83,86,87,90) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,67,70,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,71,74,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,79,82,83,86,8

/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (67,70,71,74,75,78,79,82,83,86) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,71,74,75,7

/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (83,86,87,90,91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
/var/folders/rd/x6jdq_810v96v83xb6ym8dlc0000gn/T/ipykernel_34721/3848673043.py:3: DtypeWarning: Columns (32,83,86,87,90,91,94,95,98,99,102) have mixed types.Specify dtype opt

In [184]:
query = """
SELECT *
FROM nppes
WHERE provider_business_practice_location_address_state_name = 'TN' 
AND LENGTH(provider_business_practice_location_address_postal_code) < 7
"""

In [185]:
with sqlite3.connect('data/Hop_Teaming_2018.sqlite') as db: 
    nppes_sqlite = pd.read_sql(query, db)

In [186]:
nppes_sqlite

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,1245332444,1.0,None,SORIA,SHARON,None,None,None,DMD,192 MAIN STREET,None,MANCHESTER,TN,6042.0,1223P0221X
1,1104950344,1.0,None,LYNCH,JOHN,D,DR.,II,M.D.,80 SEYMOUR STREET,None,HARTFORD,TN,6102.0,207P00000X
2,1114113875,1.0,None,SHAW,SHERRIE,BILLINGS,None,None,OTA/L,GENESIS HEALTH CARE,9 HAYWOOD AVE.,RUTLAND,TN,5701.0,224Z00000X
3,1003097320,1.0,None,GUNAWAN,JOHANNES,KEE,MR.,None,R.PH,680 CONNECTICUT AVE,None,NORWALK,TN,6854.0,183500000X
4,1043634199,1.0,None,BOWEN,HEATHER,None,None,None,None,365 MONTAUK AVE,LAWRENCE AND MEMORIAL HOSPITAL,NEW LONDON,TN,6320.0,363LP0808X


In [21]:
import numpy as np

In [37]:
taxonomy = pd.read_csv('data/nucc_taxonomy_230.csv', nrows = 100)
pd.set_option('display.max_columns', None)
taxonomy

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
95,207WX0120X,Allopathic & Osteopathic Physicians,Ophthalmology,Cornea and External Diseases Specialist,An ophthalmologist who specializes in diseases...,"Source: American Academy of Ophthalmology, www...",Cornea and External Diseases Specialist Physician,Individual
96,207WX0009X,Allopathic & Osteopathic Physicians,Ophthalmology,Glaucoma Specialist,An ophthalmologist who specializes in the trea...,"Source: American Academy of Ophthalmology, www...",Glaucoma Specialist (Ophthalmology) Physician,Individual
97,207WX0109X,Allopathic & Osteopathic Physicians,Ophthalmology,Neuro-ophthalmology,A neuro-ophthalmologist is a subspecialist of ...,"Source: American Academy of Ophthalmology, www...",Neuro-ophthalmology Physician,Individual
98,207WX0200X,Allopathic & Osteopathic Physicians,Ophthalmology,Ophthalmic Plastic and Reconstructive Surgery,A physician who specializes in oculofacial pla...,"Source: American Academy of Ophthalmology, 20...",Ophthalmic Plastic and Reconstructive Surgery ...,Individual


# Zip Codes Discrepancy

### CBSA

In [45]:
cbsa = pd.read_excel('data/ZIP_CBSA_122021.xlsx')

In [63]:
cbsa['zip_str'] = cbsa['zip'].astype('str')

In [73]:
cbsa[(cbsa['zip_str'].str.len() == 5) & (cbsa['usps_zip_pref_state'] == 'TN')]

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio,zip_str
335,37190,34980,WOODBURY,TN,0.990999,0.991435,1.0,0.991154,37190
336,37190,32660,WOODBURY,TN,0.009001,0.008565,0.0,0.008846,37190
337,37804,28940,MARYVILLE,TN,1.000000,1.000000,1.0,1.000000,37804
338,37841,99999,ONEIDA,TN,1.000000,1.000000,1.0,1.000000,37841
339,37829,28940,OAKDALE,TN,1.000000,1.000000,1.0,1.000000,37829
...,...,...,...,...,...,...,...,...,...
47151,38168,32820,MEMPHIS,TN,1.000000,1.000000,1.0,1.000000,38168
47252,38460,99999,GOODSPRING,TN,1.000000,1.000000,0.0,1.000000,38460
47253,38552,34980,CHESTNUT MOUND,TN,1.000000,1.000000,0.0,1.000000,38552
47338,38187,32820,MEMPHIS,TN,1.000000,1.000000,1.0,1.000000,38187


In [78]:
dff = cbsa[(cbsa['zip_str'].str.len() < 5)]
dff['usps_zip_pref_state'].unique()

array(['PR', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'NY', 'VI'],
      dtype=object)

In [52]:
cbsa.dtypes

zip                      int64
cbsa                     int64
usps_zip_pref_city      object
usps_zip_pref_state     object
res_ratio              float64
bus_ratio              float64
oth_ratio              float64
tot_ratio              float64
dtype: object

### NPPES ZIP CODE EXPLORATION

In [121]:
nppes_test = pd.read_csv('data/npidata_pfile_20050523-20230212.csv', 
                         usecols = [31,32,33], 
                         nrows = 9000)

In [122]:
nppes_test.columns = nppes_test.columns.str.replace(' ', '_')

In [124]:
nppes_test[nppes_test['Provider_Business_Practice_Location_Address_State_Name'] == 'TN']

,Provider_Business_Practice_Location_Address_State_Name,Provider_Business_Practice_Location_Address_Postal_Code,Provider_Business_Practice_Location_Address_Country_Code_(If_outside_U.S.)
639,TN,372032023.0,US
641,TN,376642371.0,US
684,TN,385831008.0,US
693,TN,370275250.0,US
710,TN,372031838.0,US
...,...,...,...
8783,TN,384013101.0,US
8815,TN,370120097.0,US
8849,TN,383017520.0,US
8946,TN,383053742.0,US


ZIP CODE facts:
The four extra numbers are added after a hyphen to complete the ZIP+4. So what do the extra numbers mean? These last 4 digits represent specific delivery routes within delivery areas. This extra detail means an even more precise matching to a more granular level.

NPPES business zip code isues:
1. remove '.0'
2. There are other variations of zip codes aside from having lenth of 5 and 9 